In [ ]:
# 주피터
import os

os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

In [ ]:
# 코랩
from google.colab import drive
drive.mount("/content/drive")

import os
path = '/content/drive/My Drive/projects/NLP-StockMarket/model_fin/' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

In [ ]:
!pip install konlpy   # 코랩에서만 실행

# 라이브러리

In [108]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from tqdm import tqdm
from konlpy.tag import *
from datetime import timedelta

from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, log_loss, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [109]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 종목명 선택 및 뉴스,토론방, 유튜브 데이터 통합

In [110]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [111]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col = 0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col = 0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col = 0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col = 0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col = 0)

In [112]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])

# 'Date' 타입이 int 이므로 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))

# 합쳐진 데이터들의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [113]:
# 2021년 주식 데이터가 1월 4일부터 있어서 슬라이싱
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요
640,2021-01-04,마감 국채선물 상승 111 52 증가5틱


# 주가 데이터

In [124]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(2)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24


In [125]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


In [126]:
stock_df['updown'] = 0

# 데이터 프레임 합치기

In [127]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [128]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률', 'updown']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
print(len(df))
df.head()

429597


,Date,Title,주가의 날짜,등락률,updown
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,0
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,0
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,0
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,0
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,0


# New 감성사전 load
- SP+word 합친 감성사전

In [129]:
sentiment_csv = pd.read_csv('./sentiment dictionary new.csv', index_col = 0)
sentiment_csv.head()

,Pos,Neg,Mid
0,경신,매도,책임
1,상승세,조정,랠리
2,오른,감소,유지
3,강화,전쟁,중립
4,상승,개발,산타랠리


In [130]:
pos_li = sentiment_csv['Pos'].dropna().values
mid_li = sentiment_csv['Mid'].dropna().values
neg_li = sentiment_csv['Neg'].dropna().values

# 감성 지수 계산하는 sentimental_score() 

In [136]:
### binary_sentimental_score
def binary_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['감성지수'] = 0
    
    # SP 등락률에 따른 updown 계산
    df_result.loc[df_result.query('등락률 >= 0').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < 0').index, 'updown'] = -1

    # 감성 지수는 긍정 : 1, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1').index, '감성지수'] = -1
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1').index].index) : 
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]
        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 0 :
            df_result.loc[i,'감성지수'] = 1
        else :
            df_result.loc[i,'감성지수'] = -1
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    return df_result

In [137]:
### multi_sentimental_score
def multi_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    df_result['감성지수'] = 0
    
    # SP 등락률에 따른 updown 계산
    df_result.loc[df_result.query('등락률 > 1').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < -1').index, 'updown'] = -1
    
    # 감성 지수는 긍정 : 1, 중립 : 0, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수_new'] = 999    
    
################################################################################################
    # 0  : 없음, 1: 있음
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        str_expr = f"Title.str.contains('{mid}')"
        df_result.loc[df_result.query(str_expr).index, 'Mid'] = 1
    
    
################################################################################################
    
    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 1').index, '감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result.query('Mid == 1').index, '감성지수'] = 0
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0 and Mid == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1 and Mid == 0').index, '감성지수'] = -1
    
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 0').index].index) : 
        
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]
        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 1 :
            df_result.loc[i,'감성지수'] = 1
        elif updown < -1 :
            df_result.loc[i,'감성지수'] = -1
        else :
            df_result.loc[i,'감성지수'] = 0
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    return df_result

In [138]:
df_result = binary_sentimental_score(df)

긍정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:57<00:00,  5.65it/s]


부정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1323/1323 [03:54<00:00,  5.65it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|██████████████████████████████████████████████████████████████████████████| 88853/88853 [00:55<00:00, 1598.13it/s]


In [139]:
df_result

,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,감성지수
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,1
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,1
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,1
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,1
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,1
...,...,...,...,...,...,...,...,...
354252,2022-06-29,수젠텍 원숭이두창 분자진단제품 개발 중 분자진단 플렛폼기술 확보,2022-06-30,-6.67,-1,1,1,1
354253,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,1
354254,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,1
354255,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,-1


In [140]:
df_result['updown'].value_counts()

 1    200123
-1    154134
Name: updown, dtype: int64

In [141]:
df_result['감성지수'].value_counts()

 1    237131
-1    117126
Name: 감성지수, dtype: int64

# 예측 모델 적용

In [142]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [143]:
# X : title, y : price
def data_split(X, y):
    # 수집한 데이터 읽어오기
    
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, shuffle = False, test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [144]:
X_train, X_test, y_train, y_test = data_split(df_result['Title'], df_result['감성지수'])

In [146]:
tfidf = TfidfVectorizer(tokenizer=tokenizer)

In [147]:
#  멀티 로지스틱
logistic = LogisticRegression(multi_class = 'multinomial',  random_state=0)
# logistic = LogisticRegression(random_state=0)

# lbfgs의 경우 제약조건 L2만 지원.... 뭔소리지?

# logistic = LogisticRegression(multi_class = 'multinomial', solver='saga' random_state=0)
# saga는 L1, L2 지원, 확률적경사하강법 기반   multi_class = 'ovr'... 모지?

# logistic = LogisticRegression(C=2, penalty='l2', random_state=0)

# C의 숫자가 너무 크면 과적합 (기본 1), penalty로 과적합 방지

## 파이프라인

In [148]:
pipeline = Pipeline([('tfidf',tfidf), ('clf',logistic)], verbose=True)

In [149]:
pipeline.fit(X_train, y_train)

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 4.6min
[Pipeline] ............... (step 2 of 2) Processing clf, total=  11.9s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenizer at 0x000001F2BB4AE550>)),
                ('clf',
                 LogisticRegression(multi_class='multinomial',
                                    random_state=0))],
         verbose=True)

In [150]:
y_pred_prob = pipeline.predict_proba(X_test)
y_pred = pipeline.predict(X_test)

In [151]:
y_df = df['updown'][len(X_train) : ]

## 모델 평가

In [152]:
y_pred

array([ 1,  1,  1, ...,  1, -1,  1])

In [153]:
np.unique(y_test, return_counts=True)

(array([-1,  1]), array([26171, 44681], dtype=int64))

In [154]:
np.unique(y_pred, return_counts=True)

(array([-1,  1]), array([25383, 45469], dtype=int64))

In [155]:
print(len(y_test))
print(len(y_pred))

70852
70852


In [156]:
print(f'accuracy :\t {accuracy_score(y_test, y_pred)}')

'micro', 'macro', 'weighted'
print(f'f1_score_micro :\t {f1_score(y_test, y_pred, average = "micro")}')
print(f'f1_score_macro :\t {f1_score(y_test, y_pred, average = "macro")}')
print(f'f1_score_weighted :\t {f1_score(y_test, y_pred, average = "weighted")}')

print(f'R2 \t:\t {r2_score(y_test, y_pred)}')

print(f'mse \t:\t {mean_squared_error(y_test, y_pred)}')
l_loss = log_loss(y_test, y_pred_prob)
print(f'log_loss \t:\t {l_loss}')

accuracy :	 0.8397786936148591
f1_score_micro :	 0.8397786936148591
f1_score_macro :	 0.826940120037617
f1_score_weighted :	 0.839254452969255
R2 	:	 0.3121697138498435
mse 	:	 0.6408852255405634
log_loss 	:	 0.37280606623718543


In [157]:
scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='f1_micro')
print('평균 모델 score:', scores.mean())

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.4min
[Pipeline] ............... (step 2 of 2) Processing clf, total=   8.2s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.7min
[Pipeline] ............... (step 2 of 2) Processing clf, total=   7.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.3min
[Pipeline] ............... (step 2 of 2) Processing clf, total=   7.8s
평균 모델 score: 0.8673735326437725


In [158]:
scores = classification_report(y_test, y_pred)
print(scores)

              precision    recall  f1-score   support

          -1       0.79      0.77      0.78     26171
           1       0.87      0.88      0.87     44681

    accuracy                           0.84     70852
   macro avg       0.83      0.82      0.83     70852
weighted avg       0.84      0.84      0.84     70852



## 모델 저장 및 사용

In [159]:
def save_model(model):
    with open('[Model5]pipe.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [160]:
save_model(pipeline)

저장완료


In [161]:
# 모델 사용 함수
def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('[Model5]pipe.dat','rb') as fp:     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : ')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example)*100)     # 확률값을 구해서 *100
        # 예측 결과
        r2 = pipe.predict(example)[0]     # 긍정('1'), 부정('0'), 중립('-1)
        if text == 'q':
            print("예측을 종료합니다.")
            break
        if r2 == 1:
            print(f'{stock_name} 주가가 상승할 것으로 예상됩니다.')
        elif r2 == -1:
            print(f'{stock_name} 주가가 하락할 것으로 예상됩니다.')
        else: 
            print(f'모르겠어요.....')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------')

In [31]:
model_prediction()

타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 삼성SDI 흑자에서 적자로 전환  
모르겠어요.....
확률 : 50.803
------------------------------------------------
타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : q
예측을 종료합니다.


- 넣어볼 타이틀 예시  
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충  
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현  
주식거래 뚝…"증권사, 2분기 실적 쇼크"  
삼성SDI 주식 폭망  
삼성SDI 적자 전환  
삼성SDI 흑자 전환  